In [188]:
from databaseconnection import DatabaseConnection
gds = DatabaseConnection().get_database_connection()
gds.version()

'2.3.2'

In [125]:
id = 14662
is_positive = 0
gds.run_cypher(f"MATCH (q:Question) WHERE id(q) = {id} SET q.is_positive = {is_positive}")

""


## Fixat så att position är konsekvent för QuestionAlternative

In [13]:
result = gds.run_cypher("""
MATCH (qa:QuestionAlternative)<-[:CONSISTS_OF]-(q:Question)
WHERE (q.questionCategories = ["SCHOOL_CLIMATE"] OR q.questionCategories = ["STUDENT_SCHOOL_CLIMATE"] OR q.questionCategories = ["VICTIMIZATION"] OR q.questionCategories = ["STUDENT_VICTIMIZATION"]) AND q.questionType <> "FREE_TEXT" AND NOT qa.name IN ["Jag förstår inte frågan", "Vet inte / Kan inte svara"]
RETURN q.is_positive AS is_positive, id(q) AS id, q.name, qa.name, qa.position ORDER BY q.name, qa.position
""")
for index, row in result.iterrows():
    if row["qa.position"] == 0 and row["is_positive"] == 0:
        print(row["is_positive"], row["id"], row["qa.position"], row["qa.name"], row["q.name"])

0 13625 0 Nej, det har inte hänt mig A.	Tänk på hur du har haft det i skolan och på väg till och från skolan de senaste 2-3 månaderna.  Har du känt dig ledsen, sårad, rädd eller arg, av att andra ELEVER har slagit, sparkat, knuffat, hållit fast, stängt in dig eller hotat dig med vapen?
0 7205 0 Nej, det har inte hänt mig A. Jag har under de senaste 2-3 månaderna upplevt obehag (känt mig ledsen, sårad, rädd eller arg) av att andra ELEVER har slagit, sparkat, knuffat, hållit fast, låst in eller stängt in mig i SKOLAN, på skolgården eller på väg till/från skolan
0 7221 0 Nej, det har inte hänt mig B. Jag har du under de senaste 2-3 månaderna upplevt obehag (känt mig ledsen, sårad, rädd eller arg) av att andra ELEVER har retat, kallat mig elaka saker eller hotat mig i SKOLAN, på skolgården eller på väg till/från skolan
0 13631 0 Nej, det har inte hänt mig B. Tänk på hur du har haft det i skolan och på väg till och från skolan de senaste 2-3 månaderna . Har du känt dig ledsen, sårad, rädd e

In [115]:
q_id = 13666
gds.run_cypher(f"""
MATCH (q:Question)-[:CONSISTS_OF]->(qa:QuestionAlternative)
WHERE id(q) = {q_id}
RETURN q.name, id(qa), qa.name, qa.position ORDER BY qa.position
""")

,q.name,id(qa),qa.name,qa.position
0,Vi menar att en elev blir mobbad då en annan e...,13667,"Nej, det har inte hänt mig",0
1,Vi menar att en elev blir mobbad då en annan e...,13668,"Ja, någon gång",1
2,Vi menar att en elev blir mobbad då en annan e...,13669,"Ja, ett par gånger i månaden",2
3,Vi menar att en elev blir mobbad då en annan e...,13670,"Ja, ett par gånger i veckan eller oftare",3
4,Vi menar att en elev blir mobbad då en annan e...,13671,Jag förstår inte frågan,4


In [121]:
gds.run_cypher(f"""
MATCH (q:Question)-[:CONSISTS_OF]->(qa:QuestionAlternative)
WHERE id(q) = {q_id}
RETURN q.name, id(qa), qa.name, qa.position ORDER BY qa.position
""")

,q.name,id(qa),qa.name,qa.position
0,Vi menar att en elev blir mobbad då en annan e...,13670,"Ja, ett par gånger i veckan eller oftare",0
1,Vi menar att en elev blir mobbad då en annan e...,13669,"Ja, ett par gånger i månaden",1
2,Vi menar att en elev blir mobbad då en annan e...,13668,"Ja, någon gång",2
3,Vi menar att en elev blir mobbad då en annan e...,13667,"Nej, det har inte hänt mig",3
4,Vi menar att en elev blir mobbad då en annan e...,13671,Jag förstår inte frågan,4


In [120]:
qa_id = 13670
new_position = 0
gds.run_cypher(f"""
MATCH (qa:QuestionAlternative) WHERE id(qa) = {qa_id} SET qa.position = {new_position}
""")

""


## Beräkna positivitetsIndex

In [ ]:
result = gds.run_cypher(f"""
MATCH (s:Student)-[:HAS_ANSWERED]->(a:Answer)-[:IS_ANSWER_TO]->(q:Question)-[:CONSISTS_OF]->(qa:QuestionAlternative)
WHERE (q.questionCategories = ["SCHOOL_CLIMATE"] OR q.questionCategories = ["STUDENT_SCHOOL_CLIMATE"] OR q.questionCategories = ["VICTIMIZATION"] OR q.questionCategories = ["STUDENT_VICTIMIZATION"]) AND q.questionType <> "FREE_TEXT" AND NOT qa.name IN ["Jag förstår inte frågan", "Vet inte / Kan inte svara"]
WITH s, q, q.is_positive AS is_positive, qa.position AS position
MATCH (q)-[:CONSISTS_OF]->(all_qa:QuestionAlternative)
WITH s, is_positive, position, COUNT(all_qa) - 1 AS total_positions
WITH s, is_positive, position, total_positions, toFloat(position) / total_positions AS normalized_position
WITH s, (CASE is_positive
          WHEN 1 THEN normalized_position
          ELSE 1 - normalized_position
        END) AS score
WITH id(s) AS StudentId, AVG(score) AS AvgScore
RETURN StudentId, AvgScore
ORDER BY AvgScore DESC
""")
for index, row in result.iterrows():
    print(row['StudentId'], row['AvgScore'])

In [156]:
result = gds.run_cypher(f"""
  MATCH (s:Student)-[:CHOSE_ALT]->(qa:QuestionAlternative)<-[:CONSISTS_OF]-(q:Question)
  WHERE (q.questionCategories = ["SCHOOL_CLIMATE"] OR q.questionCategories = ["STUDENT_SCHOOL_CLIMATE"] OR q.questionCategories = ["VICTIMIZATION"] OR q.questionCategories = ["STUDENT_VICTIMIZATION"]) AND q.questionType = "SINGLE_CHOICE" AND NOT qa.name IN ["Jag förstår inte frågan", "Jag förstår inte frågan ", "Vet inte / Kan inte svara", "Vet inte/Kan inte svara"]
  WITH s, q, q.is_positive AS is_positive, qa.position AS position
  MATCH (q)-[:CONSISTS_OF]->(all_qa:QuestionAlternative)
  WITH s, q, is_positive, position, collect(all_qa) AS all_qa
  WITH s, is_positive, position, size([qa IN all_qa WHERE NOT qa.name IN ["Jag förstår inte frågan", "Jag förstår inte frågan ", "Vet inte / Kan inte svara", "Vet inte/Kan inte svara"]]) - 1 AS total_positions
  WITH s, is_positive, position, total_positions, toFloat(position) / total_positions AS normalized_position
  WITH s, (CASE is_positive
              WHEN 1 THEN 1 - normalized_position
              ELSE normalized_position
            END) AS score
  WITH s, AVG(score) AS PositivityIndex
  SET s.positivityIndex = PositivityIndex
  RETURN id(s) AS StudentId, s.positivityIndex
  ORDER BY s.positivityIndex DESC
""")

for index, row in result.iterrows():
    # if row['StudentId'] ==  53885.0:
    print(row['StudentId'], row['s.positivityIndex'], sep='  \t')

152532.0  	1.0
178075.0  	1.0
35088.0  	1.0
94563.0  	1.0
161928.0  	1.0
152596.0  	1.0
163147.0  	1.0
198290.0  	1.0
230570.0  	1.0
154201.0  	1.0
200345.0  	1.0
30593.0  	1.0
105660.0  	1.0
112948.0  	1.0
192056.0  	1.0
209711.0  	1.0
27204.0  	1.0
93866.0  	1.0
149901.0  	1.0
131715.0  	1.0
189860.0  	1.0
154393.0  	1.0
118317.0  	1.0
107665.0  	1.0
65590.0  	1.0
158286.0  	1.0
34153.0  	1.0
107472.0  	1.0
81441.0  	1.0
158862.0  	1.0
166341.0  	1.0
158094.0  	1.0
159118.0  	1.0
127471.0  	1.0
173468.0  	1.0
180577.0  	1.0
31111.0  	1.0
158734.0  	1.0
180514.0  	1.0
129014.0  	1.0
155998.0  	1.0
200474.0  	1.0
158670.0  	1.0
54521.0  	1.0
161609.0  	1.0
152467.0  	1.0
109851.0  	1.0
173596.0  	1.0
199831.0  	1.0
73544.0  	0.99375
85012.0  	0.9925925925925926
94626.0  	0.9924242424242424
188135.0  	0.9914529914529914
95132.0  	0.9851851851851853
86211.0  	0.9851851851851852
53061.0  	0.985185185185185
60159.0  	0.985185185185185
68878.0  	0.984848484848485
73860.0  	0.984848484848485

In [205]:
import math

students_positivity_results = gds.run_cypher(f"""
MATCH (s:Student)
WHERE s.positivityIndex IS NOT NULL
RETURN id(s) AS StudentId, s.schoolClimateNormalityIndex AS schoolClimateNormalityIndex, s.victimizationNormalityIndex AS victimizationNormalityIndex, s.positivityIndex AS positivityIndex, s.louvainId_similarity_pos_prop AS louvainId_similarity_pos_prop
ORDER BY s.positivityIndex
""")

In [206]:
bottom_percent = math.ceil(students_positivity_results.shape[0] * 0.08)
bottom_result = students_positivity_results.head(bottom_percent)

top_percent = math.ceil(students_positivity_results.shape[0] * 0.75)
top_result = students_positivity_results.tail(top_percent)

middle_result = students_positivity_results.iloc[bottom_percent:students_positivity_results.shape[0] - top_percent]
# middle_result
bottom_result

,StudentId,schoolClimateNormalityIndex,victimizationNormalityIndex,positivityIndex,louvainId_similarity_pos_prop
0,8020,0.075660,NaN,0.000000,46
1,163210,0.072250,0.056253,0.000000,2290
2,178845,0.093908,0.067472,0.000000,94
3,200602,0.063762,0.051585,0.000000,2290
4,7957,0.058246,0.066216,0.055556,46
...,...,...,...,...,...
235,164234,0.227328,0.364833,0.482906,1418
236,28921,0.249694,0.412919,0.482906,94
237,118760,0.251215,0.348056,0.483333,1418
238,127857,0.252886,0.618269,0.483333,94


In [208]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Define custom colors for each cluster ID
# cluster_colors = {
#     46: 'tab:blue',
#     712: 'tab:orange',
#     94: 'tab:green',
#     1418: 'tab:red',
#     2232: 'tab:purple',
#     1150: 'tab:brown',
#     2290: 'tab:pink',
#     2622: 'tab:gray'
# }
cluster_colors = [
    'tab:blue',
]


bottom_percent = math.ceil(students_positivity_results.shape[0] * 0.08)
bottom_result = students_positivity_results.head(bottom_percent)
# Set a color column for bottom_result
bottom_result['color'] = 'tab:green'
rest = students_positivity_results.tail(students_positivity_results.shape[0] - bottom_percent)

# top_percent = math.ceil(students_positivity_results.shape[0] * 0.75)
# top_result = students_positivity_results.tail(top_percent)

# middle_result = students_positivity_results.iloc[bottom_percent:students_positivity_results.shape[0] - top_percent]


fig, axes = plt.subplots(5, 4, figsize=(18, 18))
for nr_of_splits in range(2, 20):
    # Split rest into nr_of_splits splits
    splits = np.array_split(rest, nr_of_splits)
    # Set a color column for each split
    for i in range(len(splits)):
        splits[i]['color'] = cluster_colors[i]
    # Concatenate bottom_result and splits
    result = pd.concat([bottom_result] + splits)
    # Plot the result
    sns.scatterplot(ax=axes[(nr_of_splits - 2) // 4, (nr_of_splits - 2) % 4], data=result, x='schoolClimateNormalityIndex', y='victimizationNormalityIndex', hue='color', palette=cluster_colors)
    axes[(nr_of_splits - 2) // 4, (nr_of_splits - 2) % 4].set_xlabel('School Climate Normality Index')
    axes[(nr_of_splits - 2) // 4, (nr_of_splits - 2) % 4].set_ylabel('Victimization Normality Index')
    axes[(nr_of_splits - 2) // 4, (nr_of_splits - 2) % 4].set_title(f'Scatterplot of Victimization vs. School Climate Normality Index with {nr_of_splits} clusters')
    axes[(nr_of_splits - 2) // 4, (nr_of_splits - 2) % 4].legend(loc='upper left')
    # plt.show()





# plt.figure(figsize=(18, 18))

# sns.scatterplot(data=bottom_result, x='schoolClimateNormalityIndex', y='victimizationNormalityIndex', color='tab:green')
# sns.scatterplot(data=middle_result, x='schoolClimateNormalityIndex', y='victimizationNormalityIndex', color='tab:blue')
# sns.scatterplot(data=top_result, x='schoolClimateNormalityIndex', y='victimizationNormalityIndex', color='tab:orange')

# plt.xlabel('School Climate Normality Index')
# plt.ylabel('Victimization Normality Index')
# plt.title('Scatterplot of Victimization vs. School Climate Normality Index')
plt.show()